In [2]:
import pandas as pd
import json
import requests
import numpy as np

import datetime
from collections import defaultdict


## the detailed api

In [13]:
URL = 'https://api.track.toggl.com/reports/api/v2/details?workspace_id=4482767&since=2020-08-1&until=2020-10-1&user_agent=email'
username = None
password = 'api_token'

cols = [
#  'id',
#  'pid',
#  'tid',
#  'uid',
 'description',
 'start',
 'end',
 'updated',
 'dur',
#  'user',
#  'use_stop',
#  'client',
 'project',
#  'project_color',
 'project_hex_color'
#  'task',
#  'billable',
#  'is_billable',
#  'cur',
#  'tags'
]


df_raw = defaultdict(list)
ind = 1
data = []
data_in = [0]
while len(data_in) != 0:
    URL = URL + '&page=' + str(ind)
    response = requests.get(URL, auth = (username, password)).json()
    data_in = response['data']
    data += data_in
    for col in cols:
        for item in data_in:
            df_raw[col].append(item[col])
    ind += 1
df = pd.DataFrame.from_dict(df_raw)
df.head(4)

,description,start,end,updated,dur,project,project_hex_color
0,ED Assignment,2020-10-01T21:00:00+05:30,2020-10-01T22:00:00+05:30,2020-10-01T22:08:44+05:30,3600000,College Work,#525266
1,Hands on ML,2020-10-01T16:00:00+05:30,2020-10-01T18:00:00+05:30,2020-10-01T20:33:23+05:30,7200000,Books and Courses,#d92b2b
2,Hands on ML,2020-10-01T13:40:00+05:30,2020-10-01T14:40:00+05:30,2020-10-01T14:59:40+05:30,3600000,Books and Courses,#d92b2b
3,Hands on ML,2020-10-01T09:55:00+05:30,2020-10-01T11:55:00+05:30,2020-10-01T13:21:48+05:30,7200000,Books and Courses,#d92b2b


In [14]:
df['start'].tolist()[0]

'2020-10-01T21:00:00+05:30'

In [15]:
def get_processed_df(df):
    df['start'] = df['start'].apply(lambda x: datetime.datetime.strptime(x.split("+")[0], '%Y-%m-%dT%H:%M:%S'))
    df['end'] = df['end'].apply(lambda x: datetime.datetime.strptime(x.split("+")[0], '%Y-%m-%dT%H:%M:%S'))
    df['updated'] = df['updated'].apply(lambda x: datetime.datetime.strptime(x.split("+")[0], '%Y-%m-%dT%H:%M:%S'))
    df['start_day'] = df['start'].apply(lambda x: x.date())
    df['end_day'] = df['end'].apply(lambda x: x.date())

    df_daily_raw = defaultdict(list)

    rows = []

    for index, row in df.iterrows():
        # if end and start day is equal
        if row['start_day'] != row['end_day']:
            row1, row2 = row.to_dict(), row.to_dict()
            start = row['start']
            end = row['end']

            end_day = row['end_day']

            mid =  (datetime.datetime.combine(row['end_day'], datetime.datetime.min.time()))

            dur1 = (mid - start).seconds * 1000
            dur2 = (end - mid).seconds * 1000
            print(dur1, dur2)
            
            row1['dur'] = dur1
            row2['dur'] = dur2

            row1['end'] = pd.Timestamp((datetime.datetime.combine(row['start_day'], datetime.datetime.max.time())))
            row2['start'] = pd.Timestamp(mid)

            # row1['start_day'] = row['start_day']
            row1['end_day'] = row1['end'].date()
            row2['start_day'] = mid.date()
            rows.append([index, row1, row2])
    print(df.columns)
    for item in rows:
        tmp = pd.DataFrame(np.insert(df.values, item[0],values = list(item[1].values()), axis = 0 ))
        tmp = pd.DataFrame(np.insert(tmp.values, item[0]+1,values = list(item[2].values()), axis = 0 ))
        tmp.drop(item[0] + 2, inplace = True)
    tmp.columns = df.columns
    return df, tmp

In [16]:
return_df, tmp = get_processed_df(df)
return_df.head()

3882000 52000
Index(['description', 'start', 'end', 'updated', 'dur', 'project',
       'project_hex_color', 'start_day', 'end_day'],
      dtype='object')


,description,start,end,updated,dur,project,project_hex_color,start_day,end_day
0,ED Assignment,2020-10-01 21:00:00,2020-10-01 22:00:00,2020-10-01 22:08:44,3600000,College Work,#525266,2020-10-01,2020-10-01
1,Hands on ML,2020-10-01 16:00:00,2020-10-01 18:00:00,2020-10-01 20:33:23,7200000,Books and Courses,#d92b2b,2020-10-01,2020-10-01
2,Hands on ML,2020-10-01 13:40:00,2020-10-01 14:40:00,2020-10-01 14:59:40,3600000,Books and Courses,#d92b2b,2020-10-01,2020-10-01
3,Hands on ML,2020-10-01 09:55:00,2020-10-01 11:55:00,2020-10-01 13:21:48,7200000,Books and Courses,#d92b2b,2020-10-01,2020-10-01
4,Hands on ML,2020-09-30 20:55:00,2020-09-30 22:10:00,2020-10-01 01:17:55,4500000,Books and Courses,#d92b2b,2020-09-30,2020-09-30


In [246]:
def get_daily_work(df):
    '''
    df should be processed 
    '''
    net_work = dict()
    for index, row in df.iterrows():
        day = row['end_day'] # doesnt matter, both days are equal
        tmp_df = df[df['end_day'] == day]
        net_work[day]  =round(sum(tmp_df['dur'].tolist())/3600000, 2) # in hrs 
    
    # tmp = get_daily_work(return_df)
    tmp = pd.DataFrame.from_dict({
        'day': list(net_work.keys()),
        'work done': list(net_work.values())
    })
        # print(tmp_df)
    return tmp

In [248]:
tmp = get_daily_work(return_df)
# tmp = pd.DataFrame.from_dict({
#     'day': list(tmp.keys()),
#     'work done': list(tmp.values())
# })
tmp.head()

,day,work done
0,2020-10-01,6.00
1,2020-09-30,5.33
2,2020-09-29,6.92
3,2020-09-28,5.32
4,2020-09-27,6.00


In [249]:
import plotly_express as px
fig = px.bar(tmp, x =  'day', y = 'work done')
fig.show()


In [263]:
return_df.head()

,description,start,end,updated,dur,project,project_hex_color,start_day,end_day
0,ED Assignment,2020-10-01 21:00:00,2020-10-01 22:00:00,2020-10-01 22:08:44,3600000,College Work,#525266,2020-10-01,2020-10-01
1,Hands on ML,2020-10-01 16:00:00,2020-10-01 18:00:00,2020-10-01 20:33:23,7200000,Books and Courses,#d92b2b,2020-10-01,2020-10-01
2,Hands on ML,2020-10-01 13:40:00,2020-10-01 14:40:00,2020-10-01 14:59:40,3600000,Books and Courses,#d92b2b,2020-10-01,2020-10-01
3,Hands on ML,2020-10-01 09:55:00,2020-10-01 11:55:00,2020-10-01 13:21:48,7200000,Books and Courses,#d92b2b,2020-10-01,2020-10-01
4,Hands on ML,2020-09-30 20:55:00,2020-09-30 22:10:00,2020-10-01 01:17:55,4500000,Books and Courses,#d92b2b,2020-09-30,2020-09-30


In [316]:
mydict = {'carl':40,
          'alan':2,
          'bob':1,
          'danny':3}
sorted(mydict)

[&#39;alan&#39;, &#39;bob&#39;, &#39;carl&#39;, &#39;danny&#39;]

In [365]:
import plotly.graph_objects as go
from collections import OrderedDict
def build_stacked_bar(df):
    '''
    returns stacked bar chart,
    input should be processed
    '''

    # there have to multiple traces equal to the number of projects
    df['project'] = ['Empty Project' if item is None else item for item in df['project'].tolist()]
    projects = list(set(df['project'].tolist()))
    days = sorted(list(set(df['end_day'].tolist())))
    trace_dict = defaultdict(dict)
    
    for project in projects:
        for day in days:
            tmp = df[df['project'] == project][df['end_day'] == day]
            if len(tmp) == 0: # no work done on that project on that day
                trace_dict[project][day] = 0
            else:
                trace_dict[project][day] = sum(tmp['dur'].tolist())/3600000
                # add zeros
    # print(trace_dict)

    trace_dict2 = dict() 
    for k, v in trace_dict.items():
        trace_dict2[k] = OrderedDict(sorted(trace_dict[k].items()))

    fig = go.Figure()
    # for project in projects:
    for key in trace_dict.keys():
        
        y = list(trace_dict[key].values())
       
        fig.add_trace(go.Bar(
            x = days,
            y = y, name = key
        ))
    fig.update_layout(width  =1000, barmode = 'stack')
    return fig,y

In [366]:
fig,y  =build_stacked_bar(return_df)
fig.show()

In [118]:
df['start'].tolist()[3].split('+')[0]

&#39;2020-10-01T09:55:00&#39;

In [119]:
cols = [
        'description',
        'start',
        'end',
        'updated',
        'dur',
        'project',
        'project_hex_color'
        ]
print(cols)

[&#39;description&#39;, &#39;start&#39;, &#39;end&#39;, &#39;updated&#39;, &#39;dur&#39;, &#39;project&#39;, &#39;project_hex_color&#39;]


In [120]:

response = requests.get(URL, auth = (username, password)).json()


In [121]:
df_tasks = defaultdict(list)
df_projects = defaultdict(list)
df = defaultdict(list)

In [122]:
data = response['data'] # is a list
for project in data:
    tasks = project['items'] # is a list too, containing dictionaries 
    for task in tasks:
        df['task'].append(task['title']['time_entry'])
        df['task time duration'].append(task['time'])
        df['project'].append(project['title']['project'])
        # df['project name'].append(project['title']['project'])
        df['project time duration'].append(project['time'])
        df['project color'].append(project['title']['hex_color'])


In [123]:
df = pd.DataFrame.from_dict(df)
pd.set_option('display.max_rows', 500)
df = df.dropna()
df.head()
# df.iloc[-1]

,task,task time duration,project,project time duration,project color
0,DLP Course 1,694000,Books and Courses,269730117,#d92b2b
1,DLS Course 2 Week 2,12879000,Books and Courses,269730117,#d92b2b
2,DLS Course 2 Week 3,18050000,Books and Courses,269730117,#d92b2b
3,DLS Course 4 Week 1,11495000,Books and Courses,269730117,#d92b2b
4,DLS Course 4 Week 2,17518117,Books and Courses,269730117,#d92b2b


In [85]:
def build_sunburst_data(df, parents, labels,values, string = 'total'):
    
    main_parents = list(set(parents))
    main_values = []
    
    for main_parent in  main_parents:
        tmp = []
        for parent, label, value in zip(parents, labels, values):
            if parent == main_parent:
                tmp.append(value)
        main_values.append(sum(tmp))
    
    parents += [string] * len(main_parents)
    labels += main_parents
    values += main_values

    parents.append("")
    labels.append(string)
    values.append(0.5*sum(values))

    return parents, labels, values

In [86]:
parents, labels, values = build_sunburst_data(df, df['project'].tolist(), df['task'].tolist(),df['task time duration'].tolist(), string = 'total')

In [94]:
import plotly.graph_objects as go
# px.sunburst(df, path=['project', 'task'], values='task time duration')
fig = go.Figure()
fig.add_trace(
    go.Sunburst(
        values = values,
        parents = parents,
        labels = labels,
        branchvalues = 'total', 
        # maxdepth = 2,
        textinfo = "none"
    )
)
fig.show()

In [14]:
df_projects = pd.DataFrame.from_dict(df_projects)
df_projects.head()

,project name,project time duration,project color
0,Books and Courses,269730117,#d92b2b
1,chores,12483000,#525266
2,College Work,162017000,#525266
3,Googling/Browsing,10759000,#06a893
4,Kaggle,194288000,#e36a00


In [16]:
df_tasks = pd.DataFrame.from_dict(df_tasks)
df_tasks.head()

,task,task time duration,project
0,DLP Course 1,694000,Books and Courses
1,DLS Course 2 Week 2,12879000,Books and Courses
2,DLS Course 2 Week 3,18050000,Books and Courses
3,DLS Course 4 Week 1,11495000,Books and Courses
4,DLS Course 4 Week 2,17518117,Books and Courses
